In [ ]:
from slmcontrol import *
import ipywidgets as widgets
from ipywidgets import interact
import matplotlib.pyplot as plt
import configparser

slm = SLMdisplay(monitor=1)

In [ ]:
config_path = 'config.ini'
config = configparser.ConfigParser()
config.read(config_path)
x,y = build_grid(config_path)
incoming = hg(x,y,0,0,config['incoming'].getfloat('waist'))

In [ ]:
xperiod = widgets.BoundedFloatText(
    value=config['grating'].getfloat('xperiod'),
    min=-100.0,
    max=100.0,
    step=0.1,
    description='x period',
    disabled=False
)

yperiod = widgets.BoundedFloatText(
    value=config['grating'].getfloat('yperiod'),
    min=-100.0,
    max=100.0,
    step=0.1,
    description='y period',
    disabled=False
)

xoffset = widgets.BoundedIntText(
    value=config['incoming'].getint('xoffset'),
    min=-10000,
    max=10000,
    description='x offset',
    disabled=False
)

yoffset = widgets.BoundedIntText(
    value=config['incoming'].getint('yoffset'),
    min=-10000,
    max=10000,
    description='y offset',
    disabled=False
)

mode = widgets.Dropdown(
    options=['Laguerre-Gauss', 'Hermite-Gauss', 'Diagonal Hermite-Gauss'],
    value='Laguerre-Gauss',
    description='Mode',
    disabled=False,
)

method = widgets.Dropdown(
    options=['bessel1', 'sinc'],
    value='bessel1',
    description='Algorithm',
    disabled=False,
)

idx1 = widgets.BoundedIntText(
    value=1,
    min=-100,
    max=100,
    description='p (m)',
    disabled=False
)

idx2 = widgets.BoundedIntText(
    value=1,
    min=-100,
    max=100,
    description='l (n)',
    disabled=False
)

waist = widgets.BoundedFloatText(
    value=0.5,
    min=.01,
    max=5.0,
    step=0.01,
    description='Waist',
    disabled=False
)

def f(mode,idx1,idx2,waist,xperiod,yperiod,xoffset,yoffset,method):
    if mode == 'Laguerre-Gauss':
        desired = lg(x, y, idx1, idx2, waist)
    elif mode == 'Hermite-Gauss':
        desired = hg(x, y, idx1, idx2, waist)
    elif mode == 'Diagonal Hermite-Gauss':
        desired = diagonal_hg(x, y, idx1, idx2, waist)
    else:
        raise ValueError("Invalid mode.")
    holo = generate_hologram(desired,incoming,x,y,config['slm'].getint('max_modulation'),xperiod,yperiod,xoffset,yoffset,method=method)
    slm.updateArray(holo)
    plt.imshow(np.abs(desired)**2,cmap='jet')

In [ ]:
interact(f,mode=mode,idx1=idx1,idx2=idx2,waist=waist,xperiod=xperiod,yperiod=yperiod,xoffset=xoffset,yoffset=yoffset,method=method);